In [4]:
#AIA Error table - set path to location in your system.
errortab='/Users/jmdunca2/ssw/sdo/aia/response/aia_V3_error_table.txt'

#Sunpy data directory (or wherever else you store your downloaded AIA data)
sunpy_dir='/Users/jmdunca2/sunpy/data/'

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

#do-dem
import lightcurves as lc
import region_fitting as rf
import nustar_dem_prep as nu
import initial_analysis as ia
import orbit_auto as oa
#import dodem

#Basics
import numpy as np
import importlib
import matplotlib.pyplot as plt
from astropy import units as u

#Time
import matplotlib.dates as mdates
import astropy.time
import datetime

#File wrangling
import glob
import pickle
import os
import pathlib

import nustar_pysolar as nustar



# from astropy.io import fits
# import nustar_pysolar as nustar
# import matplotlib.colors as colors
# from astropy.coordinates import SkyCoord
# from regions import CircleSkyRegion
import importlib
# import nustar_utilities as nuutil
import shutil


os.getcwd()

'/Users/jmdunca2/do-dem'

In [86]:
#Set path to obsid directory - initial pipeline should have been run already.
datapaths=['/Users/jmdunca2/nustar/jul-2024/21012001001/',
           '/Users/jmdunca2/nustar/jul-2024/21012006001/',
           '/Users/jmdunca2/nustar/jul-2024/21012007001/']

obsids=[dp.split('/')[-2] for dp in datapaths]
print(obsids)

#orbit 1 data interval
macro_intervals= [[astropy.time.Time('2024-07-16T11:04:00', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-16T11:30:00', scale='utc', format='isot')],
                 [astropy.time.Time('2024-07-16T19:03:30', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-16T20:00:30', scale='utc', format='isot')],
                 [astropy.time.Time('2024-07-17T04:40:00', scale='utc', format='isot'),
                    astropy.time.Time('2024-07-17T05:40:00', scale='utc', format='isot')]]
                 
           
#Orbit 1: one region
#Orbit 6: two regions
#Orbit 7: two regions

#Need AIA and NuSTAR regions:
aiaregs = [glob.glob('./*'+obsid+'*A_AIA.reg') for obsid in obsids]
nuregsA = [glob.glob('./*'+obsid+'A.reg') for obsid in obsids]
nuregsB = [glob.glob('./*'+obsid+'B.reg') for obsid in obsids]

['21012001001', '21012006001', '21012007001']


In [60]:
nuregsA

[['./AR13738__21012001001A.reg'],
 ['./AR13743__21012006001A.reg', './AR13738__21012006001A.reg'],
 ['./AR13738__21012007001A.reg', './AR13743__21012007001A.reg']]

In [19]:
mins=0.5
basepath='/Users/jmdunca2/do-dem/'

for i in range(0, len(obsids)):

    macro_interval = macro_intervals[i]

    time_intervals=[]
    checktime = macro_interval[0]
    while checktime < macro_interval[1]:
        time_intervals.append([checktime, checktime+mins*u.min])
        checktime = checktime+mins*u.min  
    count=0    
    for t in time_intervals:
        print(count, t[0].strftime('%H-%M-%S'), t[1].strftime('%H-%M-%S'))
        count+=1

    aiaregions = glob.glob('./*'+obsids[i]+'*A_AIA.reg')
    aiaregions.sort()


    for ar in range(0, len(aiaregions)):
        aiareg=aiaregions[ar]
        where=basepath+'/nccs_aia_'+obsids[i]+'_'+str(ar)
        save_path = pathlib.Path(where)
        print(save_path)
        if not save_path.exists():
            save_path.mkdir()
        print(aiareg)
        offset, rad = rf.read_regfile(aiareg, macro_interval[0], macro_interval[1], 'hourangle')
        aiadict = {'centerx': offset[0],
                   'centery': offset[1],
                   'radius': rad.value}
        oa.make_interval_dicts(time_intervals, aiadict, where=where)


    print('')


0 11-04-00 11-04-30
1 11-04-30 11-05-00
2 11-05-00 11-05-30
3 11-05-30 11-06-00
4 11-06-00 11-06-30
5 11-06-30 11-07-00
6 11-07-00 11-07-30
7 11-07-30 11-08-00
8 11-08-00 11-08-30
9 11-08-30 11-09-00
10 11-09-00 11-09-30
11 11-09-30 11-10-00
12 11-10-00 11-10-30
13 11-10-30 11-11-00
14 11-11-00 11-11-30
15 11-11-30 11-12-00
16 11-12-00 11-12-30
17 11-12-30 11-13-00
18 11-13-00 11-13-30
19 11-13-30 11-14-00
20 11-14-00 11-14-30
21 11-14-30 11-15-00
22 11-15-00 11-15-30
23 11-15-30 11-16-00
24 11-16-00 11-16-30
25 11-16-30 11-17-00
26 11-17-00 11-17-30
27 11-17-30 11-18-00
28 11-18-00 11-18-30
29 11-18-30 11-19-00
30 11-19-00 11-19-30
31 11-19-30 11-20-00
32 11-20-00 11-20-30
33 11-20-30 11-21-00
34 11-21-00 11-21-30
35 11-21-30 11-22-00
36 11-22-00 11-22-30
37 11-22-30 11-23-00
38 11-23-00 11-23-30
39 11-23-30 11-24-00
40 11-24-00 11-24-30
41 11-24-30 11-25-00
42 11-25-00 11-25-30
43 11-25-30 11-26-00
44 11-26-00 11-26-30
45 11-26-30 11-27-00
46 11-27-00 11-27-30
47 11-27-30 11-28-00
48

Going to use the DEM workflow to make lightcurves (as it nicely packages the inputs). 

In [89]:
#What instruments are you using?
#---------------------------------
aia=True
#---------------------------------
eis=False
xrt=False
#---------------------------------
plot=False
#---------------------------------
nustar=True
#If nustar is being used, here are the chosen energy ranges:
nuenergies=[[2.5,3.5],[3.5,6.], [6.,10.], [10.,15.]]
#---------------------------------

#---------------------------------
#---------------------------------
#What temperature range would you like to use? (units: log(T))
minT=5.6
maxT=7.2

#Would you prefer to plot temperatures in MK, or the default (logT)
plotMK=False
#---------------------------------
#---------------------------------

name='dems_for_lightcurves'

In [90]:
#AIA Error table - set path to location in your system.
#errortab='/Users/jmdunca2/ssw/sdo/aia/response/aia_V3_error_table.txt'

#Sunpy data directory (or wherever else you store your downloaded AIA data)
#sunpy_dir='/Users/jmdunca2/sunpy/data/'

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'

working_dir=path_to_dodem+'/jul-24-AR13743/'

import dodem

importlib.reload(oa)

ind=2

datapath=datapaths[ind]
gtifile=datapath+'event_cl/nu'+obsids[ind]+'A06_gti.fits'

suborbit_dir = path_to_dodem+'/nccs_aia_'+obsids[ind]+'_1/'

macro_interval = macro_intervals[ind]

time_intervals=[]
checktime = macro_interval[0]
while checktime < macro_interval[1]:
    time_intervals.append([checktime, checktime+mins*u.min])
    checktime = checktime+mins*u.min  
count=0    

regfile=path_to_dodem+'/AR13743__21012007001A.reg'

for time in time_intervals:
    print(time)

    data, bl, tr = oa.read_interval_dicts(time, where=suborbit_dir, bltr=True)

    dodem.dodem(time, bl, tr, xrt=xrt, aia=aia, nustar=nustar, name=name,
                                plotMK=plotMK, minT=minT, maxT=maxT,
                                plotresp=False, working_directory=working_dir,
                                default_err=0.2, path_to_dodem=path_to_dodem,
        
                                #demreg related
                                rgt_fact=1.2, max_iter=30,
                                reg_tweak=1, gloci=1, mc_in=True, mc_rounds=100, 
                                
                                #nustar related 
                                combine_fpm=True, nuenergies=nuenergies, make_nustar=True, 
                                datapath=datapath, gtifile=gtifile, regfile=regfile,
                                COM_nustar_region=False, nuclobber=False, edit_regfile=False,
        
                                #aia related
                                load_prepped_aia=data)

[<Time object: scale='utc' format='isot' value=2024-07-17T04:40:00.000>, <Time object: scale='utc' format='isot' value=2024-07-17T04:40:30.000>]
Start Time:  2024-07-17T04:40:00.000
Stop Time:  2024-07-17T04:40:30.000

Using input regfile:
/Users/jmdunca2/do-dem//AR13743__21012007001A.reg
ARF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13743/04-40-00_04-40-30/nu21012007001A06_0_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13743/04-40-00_04-40-30/nu21012007001A06_0_p_sr.rmf']
PHA File:  ['/Users/jmdunca2/do-dem//jul-24-AR13743/04-40-00_04-40-30/nu21012007001A06_0_p_sr.pha']

For NuSTAR area, using region in:  /Users/jmdunca2/do-dem//AR13743__21012007001A.reg
Max NuSTAR Energy:  10.039999961853027
Using input regfile:
/Users/jmdunca2/do-dem//AR13743__21012007001A.reg
ARF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13743/04-40-00_04-40-30/nu21012007001B06_0_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13743/04-40-00_04-40-30/nu21012007001B06_0_p_sr.rmf']
PHA File:  ['/

In [91]:
#AIA Error table - set path to location in your system.
#errortab='/Users/jmdunca2/ssw/sdo/aia/response/aia_V3_error_table.txt'

#Sunpy data directory (or wherever else you store your downloaded AIA data)
#sunpy_dir='/Users/jmdunca2/sunpy/data/'

#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'

working_dir=path_to_dodem+'/jul-24-AR13738/'

import dodem

importlib.reload(oa)

ind=2

datapath=datapaths[ind]
gtifile=datapath+'event_cl/nu'+obsids[ind]+'A06_gti.fits'

suborbit_dir = path_to_dodem+'/nccs_aia_'+obsids[ind]+'_0/'

macro_interval = macro_intervals[ind]

time_intervals=[]
checktime = macro_interval[0]
while checktime < macro_interval[1]:
    time_intervals.append([checktime, checktime+mins*u.min])
    checktime = checktime+mins*u.min  
count=0    

regfile=path_to_dodem+'/AR13738__21012007001A.reg'

for time in time_intervals:
    print(time)

    data, bl, tr = oa.read_interval_dicts(time, where=suborbit_dir, bltr=True)

    dodem.dodem(time, bl, tr, xrt=xrt, aia=aia, nustar=nustar, name=name,
                                plotMK=plotMK, minT=minT, maxT=maxT,
                                plotresp=False, working_directory=working_dir,
                                default_err=0.2, path_to_dodem=path_to_dodem,
        
                                #demreg related
                                rgt_fact=1.2, max_iter=30,
                                reg_tweak=1, gloci=1, mc_in=True, mc_rounds=100, 
                                
                                #nustar related 
                                combine_fpm=True, nuenergies=nuenergies, make_nustar=True, 
                                datapath=datapath, gtifile=gtifile, regfile=regfile,
                                COM_nustar_region=False, nuclobber=False, edit_regfile=False,
        
                                #aia related
                                load_prepped_aia=data)

[<Time object: scale='utc' format='isot' value=2024-07-17T04:40:00.000>, <Time object: scale='utc' format='isot' value=2024-07-17T04:40:30.000>]
Start Time:  2024-07-17T04:40:00.000
Stop Time:  2024-07-17T04:40:30.000

Using input regfile:
/Users/jmdunca2/do-dem//AR13738__21012007001A.reg
ARF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13738/04-40-00_04-40-30/nu21012007001A06_0_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13738/04-40-00_04-40-30/nu21012007001A06_0_p_sr.rmf']
PHA File:  ['/Users/jmdunca2/do-dem//jul-24-AR13738/04-40-00_04-40-30/nu21012007001A06_0_p_sr.pha']

For NuSTAR area, using region in:  /Users/jmdunca2/do-dem//AR13738__21012007001A.reg
Max NuSTAR Energy:  5.679999828338623
Using input regfile:
/Users/jmdunca2/do-dem//AR13738__21012007001A.reg
ARF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13738/04-40-00_04-40-30/nu21012007001B06_0_p_sr.arf']
RMF File:  ['/Users/jmdunca2/do-dem//jul-24-AR13738/04-40-00_04-40-30/nu21012007001B06_0_p_sr.rmf']
PHA File:  ['/U

In [73]:
macro_interval = macro_intervals[ind]

time_intervals=[]
checktime = macro_interval[0]
while checktime < macro_interval[1]:
    time_intervals.append([checktime, checktime+mins*u.min])
    checktime = checktime+mins*u.min  
count=0    

res_files=[]
for t in time_intervals:
    time = t
    timestring = time[0].strftime('%H-%M-%S')
    stopstring = time[1].strftime('%H-%M-%S')
    timestring=timestring+'_'+stopstring

    file = glob.glob(working_dir+'/'+timestring+'/'+timestring+'*.pickle')[0]
    res_files.append(file)

res_files

['/Users/jmdunca2/do-dem//jul-24-AR13743//04-40-00_04-40-30/04-40-00_04-40-30_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle',
 '/Users/jmdunca2/do-dem//jul-24-AR13743//04-40-30_04-41-00/04-40-30_04-41-00_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle',
 '/Users/jmdunca2/do-dem//jul-24-AR13743//04-41-00_04-41-30/04-41-00_04-41-30_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle',
 '/Users/jmdunca2/do-dem//jul-24-AR13743//04-41-30_04-42-00/04-41-30_04-42-00_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle',
 '/Users/jmdunca2/do-dem//jul-24-AR13743//04-42-00_04-42-30/04-42-00_04-42-30_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle',
 '/Users/jmdunca2/do-dem//jul-24-AR13743//04-42-30_04-43-00/04-42-30_04-43-00_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle',
 '/Users/jmdunca2/do-dem//jul-24-AR13743//04-43-00_04-43-30/04-43-00_04-43-30_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle',
 '/Users/jmdunca2/do-dem//jul-24-AR13743//04-43-30_04-44-00/04-43-30_04-44-00_5.6_7.2_dems

In [80]:
for f_ in res_files[0:2]:
    with open(f_, 'rb') as f:
        data = pickle.load(f)
    print(data.keys())
    print(data['time_interval'])
    print(data['dn_in'][-3:])

dict_keys(['time_interval', 'nuenergies', 'nufpm', 'temp_interval', 'plotMK', 'ts', 'ts_', 'trmatrix', 'dn_in', 'edn_in', 'chanax', 'DEM', 'dn_reg', 'edem', 'edn', 'chisq', 'edn_string', 'fill_color'])
[<Time object: scale='utc' format='isot' value=2024-07-17T04:40:00.000>, <Time object: scale='utc' format='isot' value=2024-07-17T04:40:30.000>]
[80862.13739093556, 59733.14130295891, 4189.165301908983]
dict_keys(['time_interval', 'nuenergies', 'nufpm', 'temp_interval', 'plotMK', 'ts', 'ts_', 'trmatrix', 'dn_in', 'edn_in', 'chanax', 'DEM', 'dn_reg', 'edem', 'edn', 'chisq', 'edn_string', 'fill_color'])
[<Time object: scale='utc' format='isot' value=2024-07-17T04:40:30.000>, <Time object: scale='utc' format='isot' value=2024-07-17T04:41:00.000>]
[76076.48023001463, 54755.53271355461, 4102.255913629722]


In [72]:
f_

['/Users/jmdunca2/do-dem//jul-24-AR13743//04-40-00_04-40-30/04-40-00_04-40-30_5.6_7.2_dems_for_lightcurves_MC_DEM_result.pickle']

In [81]:
arr = np.zeros((3, len(res_files)))

In [84]:
arr[:,0] = np.array(data['dn_in'][-3:])

In [85]:
arr

array([[76076.48023001,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.  